In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

import boto3
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
SYSTEM_MESSAGE_1 = """Your primary task is to extract invoice details from an image.
The extracted details should include the following information. If any detail is not present, use the reserved keyword "NOT_AVAILABLE":

1. Invoice Number
2. Invoice Date
3. Seller Details (Company Name, Gst No, Pan No, Address, Phone Number, Email) [if present]
4. Buyer Details (Company Name,  Gst No, Pan No, Address, Phone Number, Email) [if present]
5. Item Details (sl.no, Description, Quantity, Price) [If multiple items are present, capture all of them]
6. Total Tax (CGST, IGST, SGST) [if present]
7. Total Charges [if present]
8. Total Discount [if present]
9. Total Amount
10. Amount Paid [if present]
11. Amount Due [if present]

If the document is not an invoice, return the reserved keyword "NO_INVOICE"

Here is One Example Response format --

```
1. Invoice Number: 1118212440009383
2. Invoice Date: 21-MAR-2022
3. Seller Details:
   - Company Name: VRRDDHI FREIGHT PVT LTD
   - GST No - NOT_AVAILABLE
   - PAN No - NOT_AVAILABLE
   - Address: 28 A 116, Egato Trade Centre, New No. 318, Poonamallee High Road, Kilpauk, Chennai - 600 010, Tamil Nadu
   - Phone Number: +91 44 25003622
   - Email: seller@abc.co.in
4. Buyer Details:
   - Company Name: SUNDARAM CLAYTON LIMITED
   - GST No - NOT_AVAILABLE
   - PAN No - NOT_AVAILABLE
   - Address: PADI CHENNAI - 600050
   - Phone Number: NOT_AVAILABLE
   - Email: NOT_AVAILABLE
5. Item Details:
   - 1. Description: FREIGHT CHARGE, Quantity: 1.000, Price: 8700.00
   - 2. Description: DESTINATION CHARGES, Quantity: 1.000, Price: 1600.00
   - 3. Description: ORIGIN LOCAL CHARGES, Quantity: 1.000, Price: 4702.00
   - 4. Description: FUMIGATION CHARGES, Quantity: 1.000, Price: 3500.00
6. Total Tax:
   - CGST: 0.00
   - SGST: 0.00
   - IGST: 0.00
7. Total Charges: NOT_AVAILABLE
8. Total Discount: NOT_AVAILABLE
9. Total Amount: 889397.00
10. Amount Paid: NOT_AVAILABLE
11. Amount Due: NOT_AVAILABLE
```
"""

In [3]:
### Bedrock Model boto3 client
def get_secret_keys() -> dict:
    return {
        "aws_access_key_id": os.getenv("AWS_ACCESS_KEY"),
        "aws_secret_access_key": os.getenv("AWS_SECRET_KEY"),
        "region_name": os.getenv("REGION_NAME"),
    }


MODEL_ID = "us.meta.llama3-2-90b-instruct-v1:0"

runtime = boto3.client("bedrock-runtime", **get_secret_keys())

image_name = "C:/Users/rdas6/OneDrive/Desktop/codespace/vyturr/temp/img/Invoice-Copy-12/Page_01.png"

with open(image_name, "rb") as f:
    image = f.read()

messages = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "png", "source": {"bytes": image}}},
            {"text": SYSTEM_MESSAGE_1},
        ],
    },
]

response = runtime.converse(modelId=MODEL_ID, messages=messages)

response_text = response["output"]["message"]["content"][0]["text"]

print(response_text)

1. Invoice Number: DABUR013
2. Invoice Date: 30th Sep 2022
3. Seller Details:
   - Company Name: Vonken Technologies Pvt. Ltd.
   - GSTIN NO: 27AAFCV1618M1Z1
   - PAN NO: AAFCV1618M
   - Address: OFFICE NO.627, BOMBAY OIL SEEDS, PLOT NO.2/3/4 SECTOR-19A, VASHI NAVI MUMBAI Thane Maharashtra, India - 400705
   - Phone Number: +91-7709365614
   - Email: move@quifers.com
4. Buyer Details:
   - Company Name: DABUR INDIA LTD
   - GSTIN NO: 09AAACD0474C1Z3
   - PAN NO: AAFCV1618M
   - Address: Kaushambi Corporate Office, Kaushambi, Sahibabad, Ghaziabad, Uttar Pradesh, 201010
   - Phone Number: NOT_AVAILABLE
   - Email: NOT_AVAILABLE
5. Item Details:
   - 1. Description: Quifers TLB Quarterly License, Quantity: 1, Price: 575000
   - 2. Description: Quifers TLB Technology Infrastructure Cost, Quantity: 1, Price: 67500
6. Total Tax:
   - CGST: 0.00
   - SGST: 0.00
   - IGST: 115650
7. Total Charges: 642500
8. Total Discount: NOT_AVAILABLE
9. Total Amount: 758150
10. Amount Paid: NOT_AVAILABLE
11